In [ ]:
import pandas as pd
import re

# Load GWAS data
df = pd.read_csv("data/gwas_catalog_v1.0-associations_e114_r2025-07-21.tsv", sep="\t")

# Remove rows with missing values
df = df.dropna(subset=["SNPS", "MAPPED_GENE", "DISEASE/TRAIT", "P-VALUE"])
df = df[~df["SNPS"].isin(["NR", ""])]

# Clean base columns
df["SNPS"] = df["SNPS"].str.strip().str.upper()
df["DISEASE/TRAIT"] = df["DISEASE/TRAIT"].str.strip()

# Clean MAPPED_GENE: uppercase and remove symbols
df["MAPPED_GENE"] = df["MAPPED_GENE"].str.upper().str.strip()
df["MAPPED_GENE"] = df["MAPPED_GENE"].apply(lambda x: re.sub(r"[^A-Z0-9,\-\s|;/]", "", x))

# Replace all known separators by one single pipe `|`
df["GENE"] = df["MAPPED_GENE"].str.replace(r"\s*(,|;|/|\||\s-\s|\sAND\s|\s{2,})\s*", "|", regex=True)

# Now split on pipe
df["GENE"] = df["GENE"].str.split("|")

# Explode to one gene per row
df = df.explode("GENE")

# Final cleanup
df["GENE"] = df["GENE"].str.strip().str.upper()
df = df[df["GENE"] != ""]

# Select only studies with a p-value lower than a set threshold
threshold=5e-8
df = df[df["P-VALUE"].astype(float) < threshold]

# Drop duplicates
df = df.drop_duplicates(subset=["SNPS", "GENE", "DISEASE/TRAIT", "P-VALUE"])

# Save
df.to_csv("gwas-clean.csv", index=False)
print(f"✅ Cleaned data saved with {len(df)} rows.")
